# getterやsetterを使用しない

In [5]:
class Resitor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        
class VoltageResitance(Resitor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
        
    @property
    def voltage(self):
        return self.__voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms
        
r2 = VoltageResitance(1e3)
print(r2.current)
r2.voltage = 10
print(r2.current)

class BoundeResitance(Resitor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError("%f ohms must be > 0 " % ohms)
        self._ohms = ohms

r3 = BoundeResitance(1e3)
r3.ohms = 0 

0
0.01


ValueError: 0.000000 ohms must be > 0 

# 属性のリファクタリングの代わりに@properityを使用することを考慮に入れる

- @properityは属性の使用方法は変えずにリファクタリングできるので重宝できる

In [21]:
import datetime


class Bucket(object):
    def __init__(self, period):
        self.period_delta = datetime.timedelta(seconds=period)
        self.reset_time = datetime.datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
    
    def __repr__(self):
        return ('Bucker(max_quota=%d, quota_consumed=%d)' % (self.max_quota, self.quota_consumed))
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta
    
def fill(bucket, amount):
    now = datetime.datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount
    
def deduct(bucket, amount):
    now = datetime.datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        return False
    if bucket.quota - amount < 0:
        return False
    bucket.quota -= amount
    return True

bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
print(bucket)

if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')
print(bucket)

Bucker(max_quota=100, quota_consumed=0)
Had 99 quota
Bucker(max_quota=100, quota_consumed=99)
Not enough for 3 quota
Bucker(max_quota=100, quota_consumed=99)


# @properityを複数の属性で使用する場合はディスクリプタを使用

In [31]:
from weakref import WeakKeyDictionary

class Grade(object):
    def __init__(self):
        self._values = WeakKeyDictionary()
        
    def __get__(self, instance, instance_type):
        if instance is None : return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value

class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()
    
first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75
third_exam = Exam()
third_exam.writing_grade = 101

print(first_exam.writing_grade)
print(second_exam.writing_grade)

ValueError: Grade must be between 0 and 100

# 遅延属性(DBなどへのアクセス)には__getattr__,__getattribute__,__setattr__を使用する

In [44]:
class LazyDB(object):
    def __init__(self):
        self.exists = 5
        
    def __getattr__(self, name):
        value = 'Value for %s' % name
        setattr(self, name, value)
        return value
    
data = LazyDB()
print(data.__dict__)
print(data.foo)
print(data.__dict__)

class LoggingLazyDB(LazyDB):        
    def __getattr__(self, name):
        print('Called __getattr__(%s)' % name)
        return super().__getattr__(name)

data = LoggingLazyDB()
print(data.exists)
print(data.foo)
print(hasattr(data, 'foo'))
print(data.foo)
    
class ValidatingDB(object):
    def __init__(self):
        self.exists = 5
        
    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        try:
            return super().__getattr__(name)
        except AttributeError:
            value = 'Value for %s' % name
            setattr(self, name, value)
            return value
            
data = ValidatingDB()
print(data.exists)
print(data.foo)
print(data.foo)

class SavingDB(object):
    def __setattr__(self, name, value):
        super().__setattr__(name, value)

class LoggingSavingDB(SavingDB):
    def __setattr__(self, name, value):
        print('Called %s, %r' % (name, value))
        super().__setattr__(name, value)

data = LoggingSavingDB()
print(data.__dict__)
data.foo = 5
print(data.__dict__)
data.foo = 7
print(data.__dict__)


{'exists': 5}
Value for foo
{'foo': 'Value for foo', 'exists': 5}
5
Called __getattr__(foo)
Value for foo
True
Value for foo
Called __getattribute__(exists)
Value for exists
Called __getattribute__(foo)
Value for foo
Called __getattribute__(foo)
Value for foo
{}
Called foo, 5
{'foo': 5}
Called foo, 7
{'foo': 7}


# メタクラスによるサブクラス検証

In [45]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        if bases != (object,):
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)

class Polygon(object, metaclass=ValidatePolygon):
    sides = None
    
    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180
    
class Triangle(Polygon):
    sides = 3



# クラスの存在をメタクラスで登録する

- メタクラスは基底クラスがサブクラスされるたびに登録コードを自動チェックしてくれるため、チェックの自動化ができる

In [51]:
import json


class BetterSerializable(object):
    def __init__(self, *args):
        self.args = args
        
    def serialize(self):
        return json.dumps({'class': self.__class__.__name__,
                         'args': self.args,
                         })
    
    def __repr__(self):
        return '%s(%s)' % (self.__class__.__name__, ', '.join(str(x) for x in self.args))


registry = {}
        
def register_class(target_class):
    registry[target_class.__name__] = target_class
    
def deserialize(data):
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params['args'])
    
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        cls = type.__new__(meta, name, bases, class_dict)
        register_class(cls)
        return cls
    
class RegisteredSerializable(BetterSerializable, metaclass=Meta):
    pass

class Vector3D(RegisteredSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x, self.y, self.z = x, y, z


v3 = Vector3D(10, -7, 3)
print(v3)
data = v3.serialize()
print(data)
print(deserialize(data))

Vector3D(10, -7, 3)
{"class": "Vector3D", "args": [10, -7, 3]}
Vector3D(10, -7, 3)


# クラス属性をメタクラスで注釈する

- クラスの定義の前にクラス属性を修正可能

In [53]:
class Field(object):
    def __init__(self):
        self.name = None
        self.internal_name = None
    
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')
    
    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)
        
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        for key, value in class_dict.items():
            if isinstance(value, Field):
                value.name = key
                value.internal_name = '_' + key
        cls = type.__new__(meta, name, bases, class_dict)
        return cls

class DatabaseRow(object, metaclass=Meta):
    pass

class BetterCustomer(DatabaseRow):
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()
    
foo = BetterCustomer()
print(repr(foo.first_name), foo.__dict__)
foo.first_name = 'Euler'
print(repr(foo.first_name), foo.__dict__)

'' {}
'Euler' {'_first_name': 'Euler'}
